In [1]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
import re
import numpy as np
import time

# Test scraper on one neighborhood

In [ ]:
# /wiki/Uptown,_Chicago 4, index 9
# /wiki/Lincoln_Square,_Chicago 4, index 11
# /wiki/Rogers_Park,_Chicago 5, index 11
# /wiki/Woodlawn,_Chicago 5,  index 11


In [314]:
# create soup object
url = 'https://en.wikipedia.org/wiki/Englewood,_Chicago'
page = requests.get(url)
soup = BeautifulSoup(page.text, 'lxml')

In [315]:
# find the table in the page and put into DataFrame
table = soup.find('table', 'infobox geography vcard')
df = pd.read_html(str(table))[0]

df

,Englewood,Englewood.1
0,Community area,Community area
1,Community Area 68 – Englewood,Community Area 68 – Englewood
2,"The intersection of 63rd and Halsted, looking ...","The intersection of 63rd and Halsted, looking ..."
3,Location within the city of Chicago,Location within the city of Chicago
4,"Coordinates: .mw-parser-output .geo-default,.m...","Coordinates: .mw-parser-output .geo-default,.m..."
5,Country,United States
6,State,Illinois
7,County,Cook
8,City,Chicago
9,Neighborhoods,list EnglewoodHamilton Park


In [316]:
# location most important columns
best_cols = []
for i in df.T.iloc[0:1, :].values[0]:
    if re.search('Coordinates', str(i)) or re.search('Diploma', str(i)):
        best_cols.append(str(i))
    elif re.search('Degree', str(i)) or re.search('ZIP', str(i), flags=re.I) or re.search('Median', str(i)):
        best_cols.append(str(i))
    
best_cols

['Coordinates: .mw-parser-output .geo-default,.mw-parser-output .geo-dms,.mw-parser-output .geo-dec{display:inline}.mw-parser-output .geo-nondefault,.mw-parser-output .geo-multi-punct{display:none}.mw-parser-output .longitude,.mw-parser-output .latitude{white-space:nowrap}41°46′48″N 87°38′42″W\ufeff / \ufeff41.78000°N 87.64500°WCoordinates: 41°46′48″N 87°38′42″W\ufeff / \ufeff41.78000°N 87.64500°W',
 'Zip codes',
 'Median income']

In [288]:
len(best_cols)

3

In [289]:
df.T

,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
Portage Park,Community area,Community Area 15 - Portage Park,The southwestern entrance into Portage Park at...,Location within the city of Chicago,"Coordinates: .mw-parser-output .geo-default,.m...",Country,State,County,City,Neighborhoods,...,• White,• Black,• Hispanic,• Asian,• Other,Time zone,• Summer (DST),ZIP Codes,Median household income 2018 [1],"Source: U.S. Census, Record Information Services"
Portage Park.1,Community area,Community Area 15 - Portage Park,The southwestern entrance into Portage Park at...,Location within the city of Chicago,"Coordinates: .mw-parser-output .geo-default,.m...",United States,Illinois,Cook,Chicago,List Portage ParkJefferson ParkOld Portage Par...,...,49%,1.41%,41.57%,5.42%,2.61%,UTC-6 (CST),UTC-5 (CDT),"parts of 60630, 60634, 60641","$66,309","Source: U.S. Census, Record Information Services"


In [290]:
new = df.T
new.columns = new.iloc[:1, :].values[0]
new = new.iloc[1:]
new = new[best_cols]
new.reset_index(inplace=True)
new

,index,"Coordinates: .mw-parser-output .geo-default,.mw-parser-output .geo-dms,.mw-parser-output .geo-dec{display:inline}.mw-parser-output .geo-nondefault,.mw-parser-output .geo-multi-punct{display:none}.mw-parser-output .longitude,.mw-parser-output .latitude{white-space:nowrap}41°57′N 87°45.6′W﻿ / ﻿41.950°N 87.7600°WCoordinates: 41°57′N 87°45.6′W﻿ / ﻿41.950°N 87.7600°W",ZIP Codes,Median household income 2018 [1]
0,Portage Park.1,"Coordinates: .mw-parser-output .geo-default,.m...","parts of 60630, 60634, 60641","$66,309"


In [291]:
if len(best_cols) == 5:
    final_col_names = ['community_area', 'lat_long', 
                       'hs_diploma_or_higher', 
                       'bachelors_or_higher', 'zip_codes', 
                       'median_household_income']
elif len(best_cols) == 4:
    final_col_names = ['community_area', 'lat_long', 
                       'hs_diploma_or_higher', 'bachelors_or_higher', 
                       'zip_codes', 'median_household_income']
elif len(best_cols) == 3:
    final_col_names = ['community_area', 'lat_long',
                       'zip_codes', 'median_household_income']
else:
    final_col_names = ['community_area', 'lat_long',
                       'zip_codes']

In [292]:
# find population
for i, j in enumerate(df.T.iloc[0:1, :].values[0]):
    if re.search('Pop', j):
        print(j)
        pop_index = i + 1
        print(pop_index)

Population (2018)[1]
13


In [293]:
population = df.T.iloc[1:, pop_index:pop_index + 1].values[0][0]
population

'64954'

In [294]:
new.columns = final_col_names
new['population'] = population
new

,community_area,lat_long,zip_codes,median_household_income,population
0,Portage Park.1,"Coordinates: .mw-parser-output .geo-default,.m...","parts of 60630, 60634, 60641","$66,309",64954


In [295]:
# extract latitude and longitude
coords = new['lat_long'][0].split()[-2:]
latitude = coords[0].split('\ufeff')[1]
longitude = coords[1]
print(latitude, longitude)

41.950°N 87.7600°W


In [296]:
new['latitude'] = latitude
new['longitude'] = longitude
new.drop(columns='lat_long', inplace=True)
new

,community_area,zip_codes,median_household_income,population,latitude,longitude
0,Portage Park.1,"parts of 60630, 60634, 60641","$66,309",64954,41.950°N,87.7600°W


In [317]:
# function to do all above

def get_community_info(name):
    # create soup object
    url = 'https://en.wikipedia.org' + name
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'lxml')

    # find the table in the page and put into DataFrame
    table = soup.find('table', 'infobox geography vcard')
    df = pd.read_html(str(table))[0]
    
    # location most important columns
    best_cols = []
    for i in df.T.iloc[0:1, :].values[0]:
        if re.search('Coordinates', str(i)) or re.search('Diploma', str(i)):
            best_cols.append(str(i))
        elif re.search('Degree', str(i)) or re.search('ZIP', str(i), flags=re.I) or re.search('Median', str(i)):
            best_cols.append(str(i))
    
    new = df.T
    new.columns = new.iloc[:1, :].values[0]
    new = new.iloc[1:]
    new = new[best_cols]
    new.reset_index(inplace=True)

    # assign new column names
    if len(best_cols) == 5:
        final_col_names = ['community_area', 'lat_long', 
                           'hs_diploma_or_higher', 
                           'bachelors_or_higher', 'zip_codes', 
                           'median_household_income']
    elif len(best_cols) == 4:
        final_col_names = ['community_area', 'lat_long', 
                           'hs_diploma_or_higher', 'bachelors_or_higher', 
                           'zip_codes', 'median_household_income']
    elif len(best_cols) == 3:
        final_col_names = ['community_area', 'lat_long',
                           'zip_codes', 'median_household_income']
    else:
        final_col_names = ['community_area', 'lat_long',
                           'zip_codes']
        
    # find population
    for i, j in enumerate(df.T.iloc[0:1, :].values[0]):
        if re.search('Pop', str(j)):
            pop_index = i + 1

    population = df.T.iloc[1:, pop_index:pop_index + 1].values[0][0]
    
    new.columns = final_col_names
    new['population'] = population

    # extract latitude and longitude
    coords = new['lat_long'][0].split()[-2:]
    latitude = coords[0].split('\ufeff')[1]
    longitude = coords[1]
    
    new['latitude'] = latitude
    new['longitude'] = longitude
    new.drop(columns='lat_long', inplace=True)
    
    return new

# Get list of community areas

In [106]:
# create soup object
url = 'https://en.wikipedia.org/wiki/Community_areas_in_Chicago'
page = requests.get(url)
soup = BeautifulSoup(page.text, 'lxml')

In [107]:
table = soup.find('table', "wikitable sortable plainrowheaders")

In [108]:
pd.read_html(str(table))[0]

,Number[8],Name[8],2017[9],Area (sq mi.)[10],Area (km2),2017density (/sq mi.),2017density (/km2)
0,01,Rogers Park,55062,1.84,4.77,29925.00,11554.11
1,02,West Ridge,76215,3.53,9.14,21590.65,8336.20
2,03,Uptown,57973,2.32,6.01,24988.36,9648.06
3,04,Lincoln Square,41715,2.56,6.63,16294.92,6291.50
4,05,North Center,35789,2.05,5.31,17458.05,6740.59
...,...,...,...,...,...,...,...
73,74,Mount Greenwood,19277,2.71,7.02,7113.28,2746.45
74,75,Morgan Park,22394,3.30,8.55,6786.06,2620.11
75,76,O'Hare,12377,13.34,34.55,927.81,358.23
76,77,Edgewater,55965,1.74,4.51,32163.79,12418.51


In [109]:
# find and format links
community_links = []

for i in table.find_all('a'):
    if re.search('/wiki', str(i)):
        community_links.append(str(i).split('title=')[0].split('=')[-1].replace('"', '').replace('"', '').strip())
        
community_links

['/wiki/Rogers_Park,_Chicago',
 '/wiki/West_Ridge,_Chicago',
 '/wiki/Uptown,_Chicago',
 '/wiki/Lincoln_Square,_Chicago',
 '/wiki/North_Center,_Chicago',
 '/wiki/Lake_View,_Chicago',
 '/wiki/Lincoln_Park,_Chicago',
 '/wiki/Near_North_Side,_Chicago',
 '/wiki/Edison_Park,_Chicago',
 '/wiki/Norwood_Park,_Chicago',
 '/wiki/Jefferson_Park,_Chicago',
 '/wiki/Forest_Glen,_Chicago',
 '/wiki/North_Park,_Chicago',
 '/wiki/Albany_Park,_Chicago',
 '/wiki/Portage_Park,_Chicago',
 '/wiki/Irving_Park,_Chicago',
 '/wiki/Dunning,_Chicago',
 '/wiki/Montclare,_Chicago',
 '/wiki/Belmont_Cragin,_Chicago',
 '/wiki/Hermosa,_Chicago',
 '/wiki/Avondale,_Chicago',
 '/wiki/Logan_Square,_Chicago',
 '/wiki/Humboldt_Park,_Chicago',
 '/wiki/West_Town,_Chicago',
 '/wiki/Austin,_Chicago',
 '/wiki/West_Garfield_Park,_Chicago',
 '/wiki/East_Garfield_Park,_Chicago',
 '/wiki/Near_West_Side,_Chicago',
 '/wiki/North_Lawndale,_Chicago',
 '/wiki/South_Lawndale,_Chicago',
 '/wiki/Lower_West_Side,_Chicago',
 '/wiki/Chicago_Loop'

In [141]:
base_url = 'https://en.wikipedia.org'
for i in community_links:
    print(base_url + i)

https://en.wikipedia.org/wiki/Rogers_Park,_Chicago
https://en.wikipedia.org/wiki/West_Ridge,_Chicago
https://en.wikipedia.org/wiki/Uptown,_Chicago
https://en.wikipedia.org/wiki/Lincoln_Square,_Chicago
https://en.wikipedia.org/wiki/North_Center,_Chicago
https://en.wikipedia.org/wiki/Lake_View,_Chicago
https://en.wikipedia.org/wiki/Lincoln_Park,_Chicago
https://en.wikipedia.org/wiki/Near_North_Side,_Chicago
https://en.wikipedia.org/wiki/Edison_Park,_Chicago
https://en.wikipedia.org/wiki/Norwood_Park,_Chicago
https://en.wikipedia.org/wiki/Jefferson_Park,_Chicago
https://en.wikipedia.org/wiki/Forest_Glen,_Chicago
https://en.wikipedia.org/wiki/North_Park,_Chicago
https://en.wikipedia.org/wiki/Albany_Park,_Chicago
https://en.wikipedia.org/wiki/Portage_Park,_Chicago
https://en.wikipedia.org/wiki/Irving_Park,_Chicago
https://en.wikipedia.org/wiki/Dunning,_Chicago
https://en.wikipedia.org/wiki/Montclare,_Chicago
https://en.wikipedia.org/wiki/Belmont_Cragin,_Chicago
https://en.wikipedia.org/wiki

# Loop through all community areas and scrape information

In [320]:
# create loop to get all community area information

comms_df_list = []
try:
    for i in community_links:
        t_1 = time.time()
        print(f'Getting: {i}')
        comms_df_list.append(get_community_info(i))
        time.sleep(2)
    t_2 = time.time()
    print(f'Scraper took {int(t_2 - t_1)} minutes and {int((t_2 - t_1) % 1 * 60)} seconds.')
except:
    print(i) 


Getting: /wiki/Rogers_Park,_Chicago
Getting: /wiki/West_Ridge,_Chicago
Getting: /wiki/Uptown,_Chicago
Getting: /wiki/Lincoln_Square,_Chicago
Getting: /wiki/North_Center,_Chicago
Getting: /wiki/Lake_View,_Chicago
Getting: /wiki/Lincoln_Park,_Chicago
Getting: /wiki/Near_North_Side,_Chicago
Getting: /wiki/Edison_Park,_Chicago
Getting: /wiki/Norwood_Park,_Chicago
Getting: /wiki/Jefferson_Park,_Chicago
Getting: /wiki/Forest_Glen,_Chicago
Getting: /wiki/North_Park,_Chicago
Getting: /wiki/Albany_Park,_Chicago
Getting: /wiki/Portage_Park,_Chicago
Getting: /wiki/Irving_Park,_Chicago
Getting: /wiki/Dunning,_Chicago
Getting: /wiki/Montclare,_Chicago
Getting: /wiki/Belmont_Cragin,_Chicago
Getting: /wiki/Hermosa,_Chicago
Getting: /wiki/Avondale,_Chicago
Getting: /wiki/Logan_Square,_Chicago
Getting: /wiki/Humboldt_Park,_Chicago
Getting: /wiki/West_Town,_Chicago
Getting: /wiki/Austin,_Chicago
Getting: /wiki/West_Garfield_Park,_Chicago
Getting: /wiki/East_Garfield_Park,_Chicago
Getting: /wiki/Near_Wes

In [362]:
community_area_df_final = pd.concat(comms_df_list)
community_area_df_final.reset_index(drop=True, inplace=True)

In [363]:
community_area_df_final

,community_area,hs_diploma_or_higher,bachelors_or_higher,zip_codes,median_household_income,population,latitude,longitude
0,Rogers Park.1,84.2%,41.9%,"60626, 60645","$37,223",54402,42.0100°N,87.6700°W
1,West Ridge.1,82.17%,39.30%,60645 and parts of 60659,"$52,039",77212,42.000°N,87.6900°W
2,Uptown.1,89.86%,54.84%,"parts of 60613, 60640","$51,889",58424,41.9700°N,87.6600°W
3,Lincoln Square.1,92.96%,62.35%,"parts of 60625, 60640","$73,621[1]","41,713[1]",41.9700°N,87.6900°W
4,North Center.1,94.55%,73.61%,"parts of 60613, 60618, 60657","$115,756",35705,41.950°N,87.6800°W
...,...,...,...,...,...,...,...,...
72,Washington Heights.1,NaN,NaN,"parts of 60620, 60628 and 60643",NaN,27453,41.70383°N,87.65367°W
73,Mount Greenwood.1,NaN,NaN,60655,"$89,536[1]","18,783[1]",41.7000°N,87.7100°W
74,Morgan Park.1,NaN,NaN,parts of 60643 and 60655,"$60,747[3]","22,924[3]",41.6900°N,87.6700°W
75,O'Hare.1,88.96%,36.21%,60656 and 60666,"$49,295[1]",12377,42.0000°N,87.9200°W


# Final DataFrame formatting

In [364]:
for i in community_area_df_final['zip_codes'].values:
    print(i)

60626, 60645
60645 and parts of 60659
parts of 60613, 60640
parts of 60625, 60640
parts of 60613, 60618, 60657
parts of 60613, 60657
parts of 60614
60611, most of 60610, and parts of 60654 and 60642
part of 60631
parts of 60631, 60646, 60656
parts of 60630, 60646
parts of 60630, 60646
parts of 60625, 60646, 60659
parts of 60625, 60630
parts of 60630, 60634, 60641
parts of 60618, 60630, 60641
parts of 60634, 60635, 60707
parts of 60634, 60707
parts of 60634, 60635, 60639, 60641
parts of 60639, 60641, 60647
60618
parts of 60614, 60618, 60622, 60639, 60647
parts of 60622, 60624, 60647, and 60651.
parts of 60612, 60622, 60642, 60647
60644 (entire) 60639, 60651, 60707 (partial)
parts of 60624
parts of 60612 and 60624
parts of 60606, 60607, 60608, 60610, 60612 and 60661
parts of 60608, 60623 and 60624
parts of 60608 and 60623
parts of 60608 and 60616
60601, 60602, 60603, 60604, and parts of 60605, 60606, 60607, and 60616
parts of 60605, 60607, 60616
parts of 60609 and 60616
parts of 60609, 6

In [365]:
# clean up zip code column
community_area_df_final['zip_codes'] = community_area_df_final['zip_codes'].apply(lambda x: 
                                        x.replace(' and parts of', ',')
                                        .replace('most of ', '')
                                        .replace(' and ', ', ')
                                        .replace(' part of ', '')
                                        .replace('part of ', '')
                                        .replace('parts of ', '')
                                        .replace('(entire)', ',')
                                        .replace(' (partial)', ''))

In [366]:
community_area_df_final['community_area'] = community_area_df_final['community_area'].apply(lambda x: 
                                                                                x.replace('.1', ''))

In [367]:
community_area_df_final[['hs_diploma_or_higher', 
                         'bachelors_or_higher']] = community_area_df_final[['hs_diploma_or_higher', 
                                                                            'bachelors_or_higher']].applymap(lambda x: str(x).replace('%', ''))

In [368]:
community_area_df_final[['median_household_income', 
                         'population']] = community_area_df_final[['median_household_income', 
                                                                   'population']].applymap(lambda x: str(x).split('[')[0].replace('$', ''))

In [369]:
community_area_df_final

,community_area,hs_diploma_or_higher,bachelors_or_higher,zip_codes,median_household_income,population,latitude,longitude
0,Rogers Park,84.2,41.9,"60626, 60645","37,223",54402,42.0100°N,87.6700°W
1,West Ridge,82.17,39.30,"60645, 60659","52,039",77212,42.000°N,87.6900°W
2,Uptown,89.86,54.84,"60613, 60640","51,889",58424,41.9700°N,87.6600°W
3,Lincoln Square,92.96,62.35,"60625, 60640","73,621","41,713",41.9700°N,87.6900°W
4,North Center,94.55,73.61,"60613, 60618, 60657","115,756",35705,41.950°N,87.6800°W
...,...,...,...,...,...,...,...,...
72,Washington Heights,nan,nan,"60620, 60628, 60643",nan,27453,41.70383°N,87.65367°W
73,Mount Greenwood,nan,nan,60655,"89,536","18,783",41.7000°N,87.7100°W
74,Morgan Park,nan,nan,"60643, 60655","60,747","22,924",41.6900°N,87.6700°W
75,O'Hare,88.96,36.21,"60656, 60666","49,295",12377,42.0000°N,87.9200°W


In [370]:
community_area_df_final.to_csv('community_wiki_info.csv', index=False)

In [14]:
main_df = pd.read_csv('community_wiki_info.csv')
main_df.head()

,community_area,hs_diploma_or_higher,bachelors_or_higher,zip_codes,median_household_income,population,latitude,longitude
0,Rogers Park,84.20,41.90,"60626, 60645","37,223",54402,42.0100°N,87.6700°W
1,West Ridge,82.17,39.30,"60645, 60659","52,039",77212,42.000°N,87.6900°W
2,Uptown,89.86,54.84,"60613, 60640","51,889",58424,41.9700°N,87.6600°W
3,Lincoln Square,92.96,62.35,"60625, 60640","73,621","41,713",41.9700°N,87.6900°W
4,North Center,94.55,73.61,"60613, 60618, 60657","115,756",35705,41.950°N,87.6800°W


In [372]:
community_area_df_final.iloc[67:68, :]

,community_area,hs_diploma_or_higher,bachelors_or_higher,zip_codes,median_household_income,population,latitude,longitude
67,Englewood,nan,nan,60621,"21,275",23792,41.78000°N,87.64500°W


In [374]:
for i in community_area_df_final['zip_codes'].values:
        print(i)

60626, 60645
60645, 60659
60613, 60640
60625, 60640
60613, 60618, 60657
60613, 60657
60614
60611, 60610,, 60654, 60642
60631
60631, 60646, 60656
60630, 60646
60630, 60646
60625, 60646, 60659
60625, 60630
60630, 60634, 60641
60618, 60630, 60641
60634, 60635, 60707
60634, 60707
60634, 60635, 60639, 60641
60639, 60641, 60647
60618
60614, 60618, 60622, 60639, 60647
60622, 60624, 60647,, 60651.
60612, 60622, 60642, 60647
60644 , 60639, 60651, 60707
60624
60612, 60624
60606, 60607, 60608, 60610, 60612, 60661
60608, 60623, 60624
60608, 60623
60608, 60616
60601, 60602, 60603, 60604,, 60605, 60606, 60607,, 60616
60605, 60607, 60616
60609, 60616
60609, 60616, 60653
60653
60609
60609, 60615, 60653
60615, 60653
60609, 60615, 60621, 60637
60615, 60637
60637
60649, 60619, 60637
60619, 60620
60617, 60619
60617
60619
60617, 60619
60619, 60620, 60628
60628
60617, 60633
60617
60628, 60643
60628, 60633, 60827
60633
60629, 60632, 60638
60632
60632
60608, 60609
60608, 60609, 60616
60609
60629, 60632
60609,

# Grab neighborhood list to add to communty areas which are official names

In [2]:
# create soup object
url = 'https://en.wikipedia.org/wiki/List_of_neighborhoods_in_Chicago'
page = requests.get(url)
soup = BeautifulSoup(page.text, 'lxml')

In [6]:
soup.prettify

<bound method Tag.prettify of <!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
<head>
<meta charset="utf-8"/>
<title>List of neighborhoods in Chicago - Wikipedia</title>
<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"YBljAApAIC8AAF1hONoAAAEC","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_neighborhoods_in_Chicago","wgTitle":"List of neighborhoods in Chicago","wgCurRevisionId":996364149,"wgRevisionId":996364149,"wgArticleId":1942395,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles with short description","Short description matches Wikidata","Commons category link i

In [7]:
# find the table in the page and put into DataFrame
table = soup.find('table', 'wikitable sortable')
df = pd.read_html(str(table))[0]

df

,Neighborhood,Community area
0,Albany Park,Albany Park
1,Altgeld Gardens,Riverdale
2,Andersonville,Edgewater
3,Archer Heights,Archer Heights
4,Armour Square,Armour Square
...,...,...
241,Wildwood,Forest Glen
242,Woodlawn,Woodlawn
243,Wrightwood,Ashburn
244,Wrightwood Neighbors,Lincoln Park


In [67]:
piv_by_area = df.pivot(columns='Community area').T

In [94]:
# get neighborhoods into list
def get_neighborhood_list_by_area(x):
    row_vals = list(x)
    row_no_na = []
    for i in row_vals:
        if not (str(i) == 'nan'):
            row_no_na.append(i)

    return row_no_na

In [95]:
df = piv_by_area.T.apply(lambda x: get_neighborhood_list_by_area(x)).reset_index().iloc[:, 1:]
df

,Community area,0
0,Albany Park,"[Albany Park, Mayfair, North Mayfair, Ravenswo..."
1,Archer Heights,[Archer Heights]
2,Armour Square,"[Armour Square, Chinatown, Wentworth Gardens]"
3,Ashburn,"[Ashburn, Ashburn Estates, Beverly View, Crest..."
4,Auburn Gresham,"[Auburn Gresham, Gresham]"
...,...,...
77,West Pullman,[West Pullman]
78,West Ridge,"[Nortown, Peterson Park, Rosehill, West Ridge,..."
79,West Town,"[East Village, Noble Square, Pulaski Park, Riv..."
80,"West Town, Logan Square",[Polish Downtown]


In [65]:
row_vals = list(piv_by_area.iloc[:1, :].values[0])
row_no_na = []
for i in row_vals:
    if not (str(i) == 'nan'):
        row_no_na.append(i)

row_no_na

['Albany Park', 'Mayfair', 'North Mayfair', 'Ravenswood Manor']

In [96]:
set1 = df['Community area'].unique()

In [97]:
set2 = main_df['community_area'].values

In [109]:
common = set(set2).intersection(set(set1))
len(common)

74

In [128]:
temp1 = df.T 
temp1.columns = temp1.iloc[:1, :].values[0]
temp2 = temp1[list(common)]
temp3 = temp2.T.reset_index(drop=True)
temp3.rename(columns={'Community area': 'community_area', 0: 'neighborhood_list'}, inplace=True)
temp3

,community_area,neighborhood_list
0,Avalon Park,"[Avalon Park, Marynook, Stony Island Park]"
1,West Elsdon,[West Elsdon]
2,Archer Heights,[Archer Heights]
3,West Pullman,[West Pullman]
4,Roseland,"[Fernwood, Kensington, Lilydale, Princeton Par..."
...,...,...
69,Logan Square,"[Bucktown, Kosciuszko Park, Logan Square, Palm..."
70,Dunning,"[Belmont Heights, Belmont Terrace, Dunning, Ir..."
71,Albany Park,"[Albany Park, Mayfair, North Mayfair, Ravenswo..."
72,Gage Park,[Gage Park]


# Combine with main DataFrame

In [140]:
full = pd.concat([main_df, temp3, temp4], axis=1)
full.dropna(axis=1, inplace=True)
full

,community_area,zip_codes,population,latitude,longitude,community_area,Birth Rate,General Fertility Rate,Low Birth Weight,Prenatal Care Beginning in First Trimester,...,Prostate Cancer in Males,Stroke (Cerebrovascular Disease),Gonorrhea in Males,Tuberculosis,Below Poverty Level,Crowded Housing,Dependency,No High School Diploma,Per Capita Income,Unemployment
0,Rogers Park,"60626, 60645",54402,42.0100°N,87.6700°W,Rogers Park,16.4,62.0,11.0,73.0,...,21.7,33.7,423.3,11.4,22.7,7.9,28.8,18.1,23714,7.5
1,West Ridge,"60645, 60659",77212,42.000°N,87.6900°W,West Ridge,17.3,83.3,8.1,71.1,...,14.2,34.7,205.7,8.9,15.1,7.0,38.3,19.6,21375,7.9
2,Uptown,"60613, 60640",58424,41.9700°N,87.6600°W,Uptown,13.1,50.5,8.3,77.7,...,25.2,41.7,468.7,13.6,22.7,4.6,22.2,13.6,32355,7.7
3,Lincoln Square,"60625, 60640","41,713",41.9700°N,87.6900°W,Lincoln Square,17.1,61.0,8.1,80.5,...,27.6,36.9,195.5,8.5,9.5,3.1,25.6,12.5,35503,6.8
4,North Center,"60613, 60618, 60657",35705,41.950°N,87.6800°W,North Center,22.4,76.2,9.1,80.4,...,15.1,41.6,188.6,1.9,7.1,0.2,25.5,5.4,51615,4.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,Washington Heights,"60620, 60628, 60643",27453,41.70383°N,87.65367°W,Washington Heights,12.0,61.0,19.6,75.4,...,56.2,57.6,1274.2,3.0,15.7,1.1,42.4,15.6,19709,18.3
73,Mount Greenwood,60655,"18,783",41.7000°N,87.7100°W,Mount Greenwood,12.5,59.0,8.4,94.5,...,16.9,26.7,.,0.0,3.1,1.1,37.0,4.5,34221,6.9
74,Morgan Park,"60643, 60655","22,924",41.6900°N,87.6700°W,Morgan Park,13.2,67.5,10.6,74.5,...,39.8,47.9,741.1,2.6,13.7,0.8,39.4,10.9,26185,14.9
75,O'Hare,"60656, 60666",12377,42.0000°N,87.9200°W,O'Hare,15.8,70.0,3.5,82.0,...,2.8,40.4,.,6.3,9.5,1.9,26.5,11.0,29402,4.7


In [135]:
health_df = pd.read_csv('Public_Health_Statistics-_Selected_public_health_indicators_by_Chicago_community_area.csv')
temp4 = health_df.iloc[:, 1:]
temp4.rename(columns={'Community Area Name': 'community_area'}, inplace=True)

In [141]:
full.to_csv('chicago_final1.csv', index=False)

In [142]:
pd.read_csv('chicago_final1.csv')

,community_area,zip_codes,population,latitude,longitude,community_area.1,Birth Rate,General Fertility Rate,Low Birth Weight,Prenatal Care Beginning in First Trimester,...,Prostate Cancer in Males,Stroke (Cerebrovascular Disease),Gonorrhea in Males,Tuberculosis,Below Poverty Level,Crowded Housing,Dependency,No High School Diploma,Per Capita Income,Unemployment
0,Rogers Park,"60626, 60645",54402,42.0100°N,87.6700°W,Rogers Park,16.4,62.0,11.0,73.0,...,21.7,33.7,423.3,11.4,22.7,7.9,28.8,18.1,23714,7.5
1,West Ridge,"60645, 60659",77212,42.000°N,87.6900°W,West Ridge,17.3,83.3,8.1,71.1,...,14.2,34.7,205.7,8.9,15.1,7.0,38.3,19.6,21375,7.9
2,Uptown,"60613, 60640",58424,41.9700°N,87.6600°W,Uptown,13.1,50.5,8.3,77.7,...,25.2,41.7,468.7,13.6,22.7,4.6,22.2,13.6,32355,7.7
3,Lincoln Square,"60625, 60640","41,713",41.9700°N,87.6900°W,Lincoln Square,17.1,61.0,8.1,80.5,...,27.6,36.9,195.5,8.5,9.5,3.1,25.6,12.5,35503,6.8
4,North Center,"60613, 60618, 60657",35705,41.950°N,87.6800°W,North Center,22.4,76.2,9.1,80.4,...,15.1,41.6,188.6,1.9,7.1,0.2,25.5,5.4,51615,4.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,Washington Heights,"60620, 60628, 60643",27453,41.70383°N,87.65367°W,Washington Heights,12.0,61.0,19.6,75.4,...,56.2,57.6,1274.2,3.0,15.7,1.1,42.4,15.6,19709,18.3
73,Mount Greenwood,60655,"18,783",41.7000°N,87.7100°W,Mount Greenwood,12.5,59.0,8.4,94.5,...,16.9,26.7,.,0.0,3.1,1.1,37.0,4.5,34221,6.9
74,Morgan Park,"60643, 60655","22,924",41.6900°N,87.6700°W,Morgan Park,13.2,67.5,10.6,74.5,...,39.8,47.9,741.1,2.6,13.7,0.8,39.4,10.9,26185,14.9
75,O'Hare,"60656, 60666",12377,42.0000°N,87.9200°W,O'Hare,15.8,70.0,3.5,82.0,...,2.8,40.4,.,6.3,9.5,1.9,26.5,11.0,29402,4.7


In [144]:
full.columns

Index(['community_area', 'zip_codes', 'population', 'latitude', 'longitude',
       'community_area', 'Birth Rate', 'General Fertility Rate',
       'Low Birth Weight', 'Prenatal Care Beginning in First Trimester',
       'Preterm Births', 'Teen Birth Rate', 'Assault (Homicide)',
       'Breast cancer in females', 'Cancer (All Sites)', 'Colorectal Cancer',
       'Diabetes-related', 'Firearm-related', 'Infant Mortality Rate',
       'Lung Cancer', 'Prostate Cancer in Males',
       'Stroke (Cerebrovascular Disease)', 'Gonorrhea in Males',
       'Tuberculosis', 'Below Poverty Level', 'Crowded Housing', 'Dependency',
       'No High School Diploma', 'Per Capita Income', 'Unemployment'],
      dtype='object')